In [1]:
import requests
import json
import pandas as pd
df = pd.read_csv('pre_clean_data.csv')
df

,Unnamed: 0,name,price,address,Type,nbeds,nbath,laundry,heating,parking,air_conditioner,furnished,animal_friendly,Area
0,1,3 beds 3 baths townhouse,3300,"kelowna, bc, v1y 5t5",Townhouse,3,3,in-unit laundry,gas heating,garage parking,central ac,unfurnished,No pets,1371
1,2,2 beds 1 bath house,2000,"west kelowna, bc, v4t",House,2,1,in-unit laundry,Not specified,Not specified,Not specified,Not specified,dog and cat friendly,914
2,3,1 bed 1 bath apartment,1250,"kelowna, bc, v1v",Apartment/Condo,1,1,laundry in building,Not specified,parking available,Not specified,unfurnished,No pets,932
3,4,1 bedroom available in 2 bedroom basement suite,650,"west kelowna, bc",House,2,1,in-unit laundry,central heating,street parking,Not specified,unfurnished,No pets,1000
4,5,3 beds 1 bath house,2000,"kelowna, bc, v1v 1r1",House,3,1,Not specified,Not specified,Not specified,Not specified,Not specified,No pets,1200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1272,1335,one bedroom in 3 bed 3 bath condo,1000,"kelowna, bc, v1v 0a5",Apartment/Condo,3,3,Not specified,Not specified,Not specified,Not specified,unfurnished,No pets,932
1273,1338,come stay with us pet friendly kelowna cottage,1111,"central okanagan, bc, v1z 3r8",House,3,2,Not specified,Not specified,Not specified,Not specified,unfurnished,dog friendly,1200
1274,1342,room for rent available in rutland,850,"kelowna, bc",Room,1,1,Not specified,Not specified,Not specified,Not specified,Not specified,No pets,1100
1275,1345,room for rent for girls in rutland dm for more...,900,"kelowna, bc",House,1,1,Not specified,Not specified,Not specified,Not specified,unfurnished,No pets,1200


In [3]:
for index, row in df.iterrows():
    api_address = row['address']
    parameters = {
        "key": "ojSeJRAYm07WLKZhHKRbxXdXTiXmF6o2",
        "location": api_address
    }
    response = requests.get("http://www.mapquestapi.com/geocoding/v1/address", params=parameters)
    data = response.json()['results']
    lat = data[0]['locations'][0]['latLng']['lat']
    lng = data[0]['locations'][0]['latLng']['lng']
    df.at[index, 'lat'] = lat
    df.at[index, 'lng'] = lng

In [4]:
df

,Unnamed: 0.1,Unnamed: 0,name,price,address,Type,nbeds,nbath,laundry,heating,parking,air_conditioner,furnished,animal_friendly,Area,lat,lng
0,0,1,3 beds 3 baths townhouse,3300,"kelowna, bc, v1y 5t5",Townhouse,3,3,in-unit laundry,gas heating,garage parking,central ac,unfurnished,No pets,1371,49.87816,-119.47675
1,1,2,2 beds 1 bath house,2000,"west kelowna, bc, v4t",House,2,1,in-unit laundry,Not specified,Not specified,Not specified,Not specified,dog and cat friendly,914,49.83173,-119.62367
2,2,3,1 bed 1 bath apartment,1250,"kelowna, bc, v1v",Apartment/Condo,1,1,laundry in building,Not specified,parking available,Not specified,unfurnished,No pets,932,49.94804,-119.42824
3,3,4,1 bedroom available in 2 bedroom basement suite,650,"west kelowna, bc",House,2,1,in-unit laundry,central heating,street parking,Not specified,unfurnished,No pets,1000,49.85902,-119.58413
4,4,5,3 beds 1 bath house,2000,"kelowna, bc, v1v 1r1",House,3,1,Not specified,Not specified,Not specified,Not specified,Not specified,No pets,1200,49.91729,-119.44397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1272,1272,1335,one bedroom in 3 bed 3 bath condo,1000,"kelowna, bc, v1v 0a5",Apartment/Condo,3,3,Not specified,Not specified,Not specified,Not specified,unfurnished,No pets,932,49.93465,-119.40117
1273,1273,1338,come stay with us pet friendly kelowna cottage,1111,"central okanagan, bc, v1z 3r8",House,3,2,Not specified,Not specified,Not specified,Not specified,unfurnished,dog friendly,1200,50.11750,-119.50986
1274,1274,1342,room for rent available in rutland,850,"kelowna, bc",Room,1,1,Not specified,Not specified,Not specified,Not specified,Not specified,No pets,1100,49.88698,-119.49661
1275,1275,1345,room for rent for girls in rutland dm for more...,900,"kelowna, bc",House,1,1,Not specified,Not specified,Not specified,Not specified,unfurnished,No pets,1200,49.88698,-119.49661


In [5]:
import folium
map1 = folium.Map(
    location=[49.9138315, -119.439960],
    tiles='cartodbpositron',
    zoom_start=12,
)
df.apply(lambda row:folium.CircleMarker(location=[row["lat"], row["lng"]]).add_to(map1), axis=1)
map1

In [9]:
df.to_csv("../../processed/clean_kelowna_housing_data.csv")
df.to_excel('../../processed/clean_kelowna_housing_data.xlsx')

In [ ]:
#check